In [ ]:
import mne
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from ev_parser import gesture_parser
from functools import partial
from scipy import interpolate
from sklearn.preprocessing import StandardScaler
%matplotlib inline

path_out = 'C:/skoltech_hand_writing'
subj = ''

In [ ]:
### SCRIPT "4_File.ipynb"
def find_peaks(array_, koef=1.1, max_flg=False, mean=0, std=0 , custom_mean_flg=False):
    
    mean_=np.mean(array_)
    std_=np.std(array_)
    
    if custom_mean_flg:
        mean_=mean
        std_=std

    max_=mean_+koef*std_
    min_=mean_-koef*std_
    
    
    
    max_array_=[max_]*len(array_)
    min_array_=[min_]*len(array_)
    
    
    print(max_)
    inds=np.where(array_>max_)[0]
    
    if max_flg:
        return inds, max_
    
    else:
        return inds

def reduce_peaks(marks_array_inds,  threhold, with_log=False,):
    
    new_marks_array_inds=[marks_array_inds[0]]
    for i in range(1,len(marks_array_inds)):
        if marks_array_inds[i]-new_marks_array_inds[-1]<threhold:
            if with_log:
                print(i)
        else:
            new_marks_array_inds.append(marks_array_inds[i])

    return new_marks_array_inds

def reject_list_by_sigma_2(epochs_1):

    data_epochs_1=epochs_1.get_data()
    mean_=np.mean(data_epochs_1,(0,2))
    std_=np.std(data_epochs_1,(0,2))

    max_=(mean_+7*std_)
    min_=(mean_-7*std_)



    axx=data_epochs_1.shape[0]
    axx2=data_epochs_1.shape[2]
    rej=[]

    for i in range(axx):
        for j in range(axx2):
            if np.sum(max_-data_epochs_1[i,:,j]<0)!=0 or np.sum(min_-data_epochs_1[i,:,j]>0)!=0:
                  rej.append(i)

        reject_list=np.unique(rej).tolist()
    return reject_list

In [ ]:
codes=np.arange(1, 6)

handwriting=[]
eeg=[]
marks=[]
sensor=[]
sess=[]

sh_list = []
for code in codes:
    events_2_3 = np.loadtxt(f'{path_out}/hand_writing/{subj}/epochs/Rec_{code}/events_2-3_tablet_data.csv')
                           

    # CHECK THE TIME CONSTANT IN THE FOLLOWING LINE
    sh = (events_2_3[events_2_3[:, 2]==-3][:, 0] - events_2_3[events_2_3[:, 2]==-2][:, 0]) < 2.5 * 2048.0
    sh_list = np.concatenate([sh_list, sh])

    # TAKE FROM THE [1:] EPOCH! BECAUSE THERE IS NO MARK FOR THE FIRST ONE!
    # ALSO LOOK AT THE SESS::: sess.append(...).iloc[1:] !!! also taken from the second one!!!!

    e1=mne.read_epochs(f'{path_out}/hand_writing/{subj}/epochs/Rec_{code}/epochs_handwriting_tablet-epo.fif', verbose = False)
    e2=mne.read_epochs(f'{path_out}/hand_writing/{subj}/epochs/Rec_{code}/epochs-long-epo.fif', verbose = False)#[1:]
    e3=mne.read_epochs(f'{path_out}/hand_writing/{subj}/epochs/Rec_{code}/epochs-marks-long-epo.fif', verbose = False)#[1:]
    e4=e3.copy().drop_channels(['C'])
    e4._data = e3._data[:, 0, None, :]
    e4._data = e3._data[:, 1, None, :]
    print(f'HANDWRITING EPOCHS: {len(e1)}')
    print(f'EPOCHS: {len(e2)}')
    print(f'MARKS: {len(e3)}')    
    sess.append(pd.read_excel(f'8_Sessions/Digits_Tyumen_{code}.xlsx').iloc[1:] )

    handwriting.append(e1)
    eeg.append(e2)
    marks.append(e3)
    sensor.append(e4)

sess_df=pd.concat(sess).reset_index()[['Digit_text']]
print(sess_df.shape, sess_df.shape[0] / 5)
sess_df.head()

In [ ]:
marks_all=mne.concatenate_epochs(marks)
sensor_all=mne.concatenate_epochs(sensor)
eeg_all=mne.concatenate_epochs(eeg)
handwriting_all=mne.concatenate_epochs(handwriting)

# drop epocs based on signal dispersion
drop_list=reject_list_by_sigma_2(eeg_all)

marks_all.drop(drop_list)
sensor_all.drop(drop_list)
eeg_all.drop(drop_list)
handwriting_all.drop(drop_list)
sess_df=sess_df.drop(drop_list, axis=0).reset_index()[['Digit_text']]
sh_list = np.delete(sh_list, drop_list)

In [ ]:
epochs_num=len(eeg_all)
epochs_duration=eeg_all._data.shape[-1]

ss=StandardScaler()

marks_data=np.squeeze(marks_all._data[:,1,:])
marks_data_sq=marks_data*10
marks_data_sc=ss.fit_transform(marks_data.flatten().reshape(-1,1)).reshape(epochs_num,epochs_duration)

print(marks_data.shape, marks_data_sc.shape)

marks_data_sc=marks_data_sc.reshape(epochs_num,1,epochs_duration)
sensor_all._data=marks_data_sc

In [ ]:
# Drop epochs based on trial length
new_drop_list = np.where(sh_list == 0)[0]
marks_all.drop(new_drop_list)
sensor_all.drop(new_drop_list)
eeg_all.drop(new_drop_list)
handwriting_all.drop(new_drop_list)
sess_df=sess_df.drop(new_drop_list, axis=0).reset_index()[['Digit_text']]

sess_df.to_csv(f'{path_out}/hand_writing/{subj}/Digits_trials_final_2.xlsx')

In [ ]:
def find_peaks(array_, koef=1.5, max_flg=False, mean=0, std=0 , custom_mean_flg=False):
    
    mean_=np.mean(array_)
    std_=np.std(array_)
    
    if custom_mean_flg:
        mean_=mean
        std_=std

    max_=mean_+koef*std_
    
    print(max_)
    inds=np.where(array_>max_)[0]
    
    if max_flg:
        return inds, max_
    
    else:
        return inds

In [ ]:
# plot averaged marks and sensor data to check the alignment
os.makedirs(f'{path_out}/hand_writing/{subj}/output', exist_ok=True)

mean=np.average(marks_all._data,0)
mean_B=mean[0]
mean_C=mean[1]

plt.plot(mean_B, label = 'channel B')
plt.plot(mean_C, label = 'channel C')
plt.legend()
plt.savefig(f'{path_out}/hand_writing/{subj}/output/channels BC.png', dpi=300, bbox_inches = 'tight')

In [ ]:
# plot ERP for each channel
for i in eeg_all.ch_names:
    ind=eeg_all.ch_names.index(i)
    fig = eeg_all.plot_image([ind])
    fig[0].savefig(f'{path_out}/hand_writing/{subj}/output/{i}.png', dpi = 300)

In [ ]:
# plot "ERP" of sensor channel
fig = sensor_all.copy().crop(-2,4).plot_image([0])
fig[0].savefig(f'{path_out}/hand_writing/{subj}/output/sensor.png', dpi = 300)

In [ ]:
# plot "ERP" of marks channel
fig = marks_all.copy().crop(-2,4).plot_image([0])
fig[0].savefig(f'{path_out}/hand_writing/{subj}/output/marks.png', dpi = 300)

In [ ]:
# plot point whithin each epoch, where the actual drawing started based on the sensor recordings
%matplotlib inline
mean=np.average(marks_data,0)
mean_sq=np.average(marks_data_sq,0)
mean_sc=np.average(marks_data_sc,0)

print(mean.shape,mean_sq.shape)

plt.plot(mean)
plt.plot(mean_sq)
plt.plot(mean_sc)